# Import packages

In [1]:
from deepface import DeepFace
from common_functionality import *
import numpy as np
import contextlib
import os
import sys
import pandas as pd
from scipy.stats import entropy
import cv2
import json

In [2]:
@contextlib.contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

# Saving and Prediction functionality

## Emotion Classification

In [3]:
# Function which writes dominant emotion for a given image
def write_emotion_detected(data_path, base_img_name, roi_name, emotion_det):
    with open(data_path, 'r+') as f:
        processed_data = json.load(f)
        if 'frames_list' in processed_data.keys():
            for index_f, frame in enumerate(processed_data['frames_list']):
                if frame['frame_num'] == base_img_name:
                    frame_dict = frame.copy()
                    frame_pos = index_f
                    break
                
            for index_r, roi in enumerate(processed_data['frames_list'][frame_pos]['rois_list']):
                if roi['roi_num'] == roi_name:
                    roi_dict = roi.copy()
                    roi_pos = index_r
                    break
            
            #print("Before:", roi_dict)
            roi_dict['emotion_classified'] = emotion_det
            #print("After:",roi_dict)

            processed_data['frames_list'][frame_pos]['rois_list'][roi_pos] = roi_dict
            f.seek(0)
            json.dump(processed_data, f, indent=4)
            f.truncate()  # This prevents leftover characters from previous write

# Function which writes emotion probabilities for a given image
def write_emotion_probs(data_path, base_img_name, roi_name, emotion_probs):
    with open(data_path, 'r+') as f:
        processed_data = json.load(f)
        if 'frames_list' in processed_data.keys():
            for index_f, frame in enumerate(processed_data['frames_list']):
                if frame['frame_num'] == base_img_name:
                    frame_dict = frame.copy()
                    frame_pos = index_f
                    break
                
            for index_r, roi in enumerate(processed_data['frames_list'][frame_pos]['rois_list']):
                if roi['roi_num'] == roi_name:
                    roi_dict = roi.copy()
                    roi_pos = index_r
                    break
            
            #print("Before:", roi_dict)
            roi_dict['emotion_probabilities'] = emotion_probs
            #print("After:",roi_dict)

            processed_data['frames_list'][frame_pos]['rois_list'][roi_pos] = roi_dict
            f.seek(0)
            json.dump(processed_data, f, indent=4)
            f.truncate()  # This prevents leftover characters from previous write

# Function which writes entropy from emotion probabilities for a given image
def write_emotion_entropy(data_path, base_img_name, roi_name, entropy_val):
    with open(data_path, 'r+') as f:
        processed_data = json.load(f)
        if 'frames_list' in processed_data.keys():
            for index_f, frame in enumerate(processed_data['frames_list']):
                if frame['frame_num'] == base_img_name:
                    frame_dict = frame.copy()
                    frame_pos = index_f
                    break
                
            for index_r, roi in enumerate(processed_data['frames_list'][frame_pos]['rois_list']):
                if roi['roi_num'] == roi_name:
                    roi_dict = roi.copy()
                    roi_pos = index_r
                    break
            
            #print("Before:", roi_dict)
            roi_dict['entropy'] = entropy_val
            #print("After:",roi_dict)

            processed_data['frames_list'][frame_pos]['rois_list'][roi_pos] = roi_dict
            f.seek(0)
            json.dump(processed_data, f, indent=4)
            f.truncate()  # This prevents leftover characters from previous write

# Function which writes predicted arousal and valence values for a given image
def write_av_values(data_path, base_img_name, roi_name, arousal_value, valence_value):
    with open(data_path, 'r+') as f:
        processed_data = json.load(f)
        if 'frames_list' in processed_data.keys():
            for index_f, frame in enumerate(processed_data['frames_list']):
                if frame['frame_num'] == base_img_name:
                    frame_dict = frame.copy()
                    frame_pos = index_f
                    break
                
            for index_r, roi in enumerate(processed_data['frames_list'][frame_pos]['rois_list']):
                if roi['roi_num'] == roi_name:
                    roi_dict = roi.copy()
                    roi_pos = index_r
                    break
            
            #print("Before:", roi_dict)
            roi_dict['predicted_arousal'] = arousal_value
            roi_dict['predicted_valence'] = valence_value
            #print("After:",roi_dict)

            processed_data['frames_list'][frame_pos]['rois_list'][roi_pos] = roi_dict
            f.seek(0)
            json.dump(processed_data, f, indent=4)
            f.truncate()  # This prevents leftover characters from previous write

In [4]:
# Function which predicts the dominant emotion displayed in a given image
def predict_emotion(snapshot_path, snapshot_img_name, vid_emotion_categories, backend_detector):
    with suppress_stdout():
        result = DeepFace.analyze(snapshot_path, actions=['emotion'], detector_backend=backend_detector, enforce_detection=False)
    #result = DeepFace.analyze(snapshot_path, actions=['emotion'], detector_backend="yolov8", enforce_detection=False)

    # Initialise emotion category dictionary if empty
    if not bool(vid_emotion_categories):
        for category in result[0]['emotion'].keys():
            vid_emotion_categories[category] = []
    #         os.mkdir(os.path.join(imgs_path, category))
    emotion_displayed = result[0]['dominant_emotion']
    emotion_probs = result[0]['emotion']
    vid_emotion_categories[emotion_displayed].append(snapshot_img_name)
    emotion_probs_df = pd.DataFrame([emotion_probs])
    entropy_value = emotion_probs_df.apply(entropy, axis=1).iloc[0]

    return emotion_displayed, emotion_probs, entropy_value

# Function which computes arousal and valence from predicted emotion probabilities from a given image
def calculate_av(emotion_probs):
    emotion_av_values = {
        'happy': {'valence': 0.8, 'arousal': 0.7},
        'sad': {'valence': -0.6, 'arousal': -0.4},
        'neutral': {'valence': 0.0, 'arousal': 0.0},
        'angry': {'valence': -0.7, 'arousal': 0.6},
        'fear': {'valence': -0.6, 'arousal': 0.7},
        'surprise': {'valence': 0.4, 'arousal': 0.8},
        'disgust': {'valence': -0.7, 'arousal': 0.3}
    }

    arousal_predicted = 0
    valence_predicted = 0

    for emotion, prob in emotion_probs.items():
        if emotion in emotion_av_values:
            arousal_predicted += prob/100 * emotion_av_values[emotion]['arousal']
            valence_predicted += prob/100 * emotion_av_values[emotion]['valence']

    return arousal_predicted, valence_predicted

## Person Classification


In [5]:
# Function which writes detected person label for a given image
def write_person_detected(data_path, base_img_name, roi_name, person_det):
    with open(data_path, 'r+') as f:
        processed_data = json.load(f)
        if 'frames_list' in processed_data.keys():
            for index_f, frame in enumerate(processed_data['frames_list']):
                if frame['frame_num'] == base_img_name:
                    frame_dict = frame.copy()
                    frame_pos = index_f
                    break
                
            for index_r, roi in enumerate(processed_data['frames_list'][frame_pos]['rois_list']):
                if roi['roi_num'] == roi_name:
                    roi_dict = roi.copy()
                    roi_pos = index_r
                    break
            
            #print("Before:",roi_dict)
            roi_dict['person_classified'] = person_det
            #print("After:", roi_dict)

            processed_data['frames_list'][frame_pos]['rois_list'][roi_pos] = roi_dict
            f.seek(0)
            json.dump(processed_data, f, indent=4)
            f.truncate()  # This prevents leftover characters from previous write

# Function which writes person classification distance for a given image
def write_person_score(data_path, base_img_name, roi_name, person_score):
    with open(data_path, 'r+') as f:
        processed_data = json.load(f)
        if 'frames_list' in processed_data.keys():
            for index_f, frame in enumerate(processed_data['frames_list']):
                if frame['frame_num'] == base_img_name:
                    frame_dict = frame.copy()
                    frame_pos = index_f
                    break
                
            for index_r, roi in enumerate(processed_data['frames_list'][frame_pos]['rois_list']):
                if roi['roi_num'] == roi_name:
                    roi_dict = roi.copy()
                    roi_pos = index_r
                    break
            
            #print("Before:",roi_dict)
            roi_dict['person_score'] = person_score
            #print("After:", roi_dict)

            processed_data['frames_list'][frame_pos]['rois_list'][roi_pos] = roi_dict
            f.seek(0)
            json.dump(processed_data, f, indent=4)
            f.truncate()  # This prevents leftover characters from previous write



In [6]:
# Function which compares interviewer and interviewee images and assigns person label
def compare_person_new(snapshot_to_compare, interviewer_refs, interviewee_refs, facial_rec_model, custom_threshold='auto'):
    best_match = "Unknown"
    best_distance = float('inf')

    # Compare with interviewer
    cat1, dist1 = get_best_match(snapshot_to_compare, interviewer_refs, "Interviewer", facial_rec_model, threshold=custom_threshold)
    
    # Compare with interviewee
    cat2, dist2 = get_best_match(snapshot_to_compare, interviewee_refs, "Interviewee", facial_rec_model, threshold=custom_threshold)

    # Decide best match
    if dist1 is not None and dist2 is not None:
        if dist1 < dist2:
            best_match, best_distance = cat1, dist1
        else:
            best_match, best_distance = cat2, dist2
    elif dist1 is not None:
        best_match, best_distance = cat1, dist1
    elif dist2 is not None:
        best_match, best_distance = cat2, dist2
    else:
        best_match = "Unknown"

    return best_match, best_distance if best_match != "Unknown" else None


# Helper to process all views for one person
def get_best_match(snapshot, reference_imgs, category_label, model, threshold='auto'):
    min_dist = float('inf')
    matched = False

    for ref in reference_imgs:
        try:
            with suppress_stdout():
                if threshold == 'auto':
                    result = DeepFace.verify(snapshot, ref, model_name=model, detector_backend='mtcnn', distance_metric='euclidean_l2', silent=True)
                else:
                    result = DeepFace.verify(snapshot, ref, model_name=model, detector_backend='mtcnn', distance_metric='euclidean_l2', threshold=threshold, silent=True)
            #print(f"{category_label} comparison: {result}")
            if result["verified"] and result["distance"] < min_dist:
                min_dist = result["distance"]
                matched = True
        except Exception as e:
            #print(f"Error comparing with {category_label} reference: {e}")
            continue
    
    return (category_label if matched else "Unknown", min_dist if matched else None)


# Other functionality

In [7]:
# Function which returns list of all ROIs under which each person label in a given video is assigned to 
def load_person_categories(data_path):
    person_categories = dict()
    with open(data_path, 'r+') as f:
        processed_data = json.load(f)
        if 'frames_list' in processed_data.keys():
            for index_f, frame in enumerate(processed_data['frames_list']):              
                for index_r, roi in enumerate(processed_data['frames_list'][index_f]['rois_list']):
                    if 'person_classified' in roi.keys():
                        if roi['person_classified'] not in person_categories.keys():
                            person_categories[roi['person_classified']] = []
                        person_categories[roi['person_classified']].append(roi['roi_num']+".png")
            
    
    return person_categories

# Function which returns list of all ROIs under which each emotion label in a given video is assigned to 
def load_emotion_categories(data_path):
    emotion_categories = dict()
    with open(data_path, 'r+') as f:
        processed_data = json.load(f)
        if 'frames_list' in processed_data.keys():
            for index_f, frame in enumerate(processed_data['frames_list']):              
                for index_r, roi in enumerate(processed_data['frames_list'][index_f]['rois_list']):
                    if 'emotion_classified' in roi.keys():
                        if roi['emotion_classified'] not in emotion_categories.keys():
                            emotion_categories[roi['emotion_classified']] = []
                        emotion_categories[roi['emotion_classified']].append(roi['roi_num']+".png")
            
    
    return emotion_categories

## Select ROIs database folder (Jon Mallia or Mark Lawrence Zammit)

In [8]:
#vid_frames_folders = [folder for folder in os.listdir(".") if os.path.isdir(os.path.join(".", folder)) and folder.startswith("vid_frames")]

rois_folders = [folder for folder in os.listdir(".") if os.path.isdir(os.path.join(".", folder)) and folder.startswith("face_rois_dt")]

for index, rois_dir in enumerate(rois_folders):
    print(f'{index+1}: {rois_dir}')

1: face_rois_dt
2: face_rois_dt_ml


In [9]:
valid_choice = False
while not valid_choice:
    index_choice = input('Enter index of facial ROIs folder: ')

    if index_choice.isnumeric():
        if int(index_choice) > len(rois_folders) or int(index_choice) <= 0:
            valid_choice = False
            print('Invalid choice!')
        else:
            valid_choice = True
            #frames_dir = vid_frames_folders[int(index_choice)-1]
            rois_dir_chosen = rois_folders[int(index_choice)-1]
            #rois_dir_chosen = f'face_rois_dt_{"_".join(frames_dir.split("_")[2:])}' if frames_dir != "vid_frames" else "face_rois_dt"
            video_data_path = os.path.join('data', f'video_data_{"_".join(rois_dir_chosen.split("_")[-1:])}') if rois_dir_chosen != "face_rois_dt" else os.path.join('data', 'video_data')
            interviewee_data_path = 'data/interviewee_names_data_ml.json' if rois_dir_chosen != "face_rois_dt" else 'data/interviewee_names_data.json'
            interviewer_data_path = 'data/interviewer_names_data_ml.json' if rois_dir_chosen != "face_rois_dt" else 'data/interviewer_names_data.json'

            print("ROIs Folder selected:", rois_dir_chosen)
            #print(rois_dir_chosen)
            print(video_data_path)

    else:
        valid_choice = False
        print("Index entered is not numeric!")

ROIs Folder selected: face_rois_dt
data\video_data


## Load interviewer and interviewee names from their respective data files

In [10]:
if os.path.exists(interviewee_data_path):
    with open(interviewee_data_path, 'r') as f:
        interviewee_data = json.load(f)

if os.path.exists(interviewer_data_path):
    with open(interviewer_data_path, 'r') as f:
        interviewer_data = json.load(f)

# Asking user for directory of video to process

In [11]:
frames_subDirs = []

print('List of available of videos with extracted ROIs:')
for root, dirs, files in os.walk(rois_dir_chosen, topdown=True):
    for subDir in dirs:
        frames_subDirs.append(subDir)

frames_subDirs_sorted = sorted(frames_subDirs, key=lambda x: int(x.split('_')[-1]))

for index, subDir in enumerate(frames_subDirs_sorted):
    print(f'{index+1}: {os.path.join(rois_dir_chosen, subDir)}')

List of available of videos with extracted ROIs:
1: face_rois_dt\video_1
2: face_rois_dt\video_2
3: face_rois_dt\video_3
4: face_rois_dt\video_4
5: face_rois_dt\video_5
6: face_rois_dt\video_6
7: face_rois_dt\video_7
8: face_rois_dt\video_8
9: face_rois_dt\video_9
10: face_rois_dt\video_10
11: face_rois_dt\video_11
12: face_rois_dt\video_12
13: face_rois_dt\video_13
14: face_rois_dt\video_14
15: face_rois_dt\video_15
16: face_rois_dt\video_16
17: face_rois_dt\video_17
18: face_rois_dt\video_18
19: face_rois_dt\video_19
20: face_rois_dt\video_20
21: face_rois_dt\video_21
22: face_rois_dt\video_22


In [12]:
valid_choice = False
dir_chosen = ""
dir_chosen_full_path = ""
while not valid_choice:
    index_choice = input('Enter index of directory that you want to perform classification: ')

    if index_choice.isnumeric():
        if int(index_choice) > len(frames_subDirs_sorted) or int(index_choice) <= 0:
            valid_choice = False
            print('Invalid choice!')
        else:
            valid_choice = True
            dir_chosen = frames_subDirs_sorted[int(index_choice)-1]
            dir_chosen_full_path = os.path.join(rois_dir_chosen, dir_chosen)
            chosen_video_data_path = os.path.join(video_data_path, f'{dir_chosen}.json')
            print("Directory selected:", dir_chosen_full_path)
    else:
        valid_choice = False
        print("Index entered is not numeric!")

Directory selected: face_rois_dt\video_14


# Validate if video data exists

In [13]:
# Check if data file for selected video exists in the data folder
video_data_file_exists = check_if_data_file_exists(dir_chosen)

if not video_data_file_exists:
    raise Exception(f'Data file for {dir_chosen} does not exist in {video_data_path}!')
    exit(0)

print(chosen_video_data_path)

# Check if there is a list of extracted ROIs in the selected video data file
roi_list_exists_in_file = validate_roi_list_exists(chosen_video_data_path)

if not roi_list_exists_in_file:
    raise Exception(f'Please ensure that all facial ROIs from {dir_chosen} have been extracted before running this!')
    exit(0)

print("Chosen video data file is valid!")

data\video_data\video_14.json
Chosen video data file is valid!


# Main execution

## Initialisation of Parameters for Reviewing Results after Execution

In [14]:
video_emotions_categorized = dict()
video_person_categorized = dict()

In [15]:
print([file for file in os.listdir(dir_chosen_full_path)])  # TESTING PURPOSES

['frame_0009_00.png', 'frame_0010_00.png', 'frame_0011_00.png', 'frame_0012_00.png', 'frame_0013_00.png', 'frame_0014_00.png', 'frame_0015_00.png', 'frame_0016_00.png', 'frame_0016_01.png', 'frame_0017_00.png', 'frame_0018_00.png', 'frame_0019_00.png', 'frame_0020_00.png', 'frame_0020_01.png', 'frame_0021_00.png', 'frame_0021_01.png', 'frame_0022_00.png', 'frame_0023_00.png', 'frame_0024_00.png', 'frame_0025_00.png', 'frame_0026_00.png', 'frame_0027_00.png', 'frame_0028_00.png', 'frame_0029_00.png', 'frame_0030_00.png', 'frame_0031_00.png', 'frame_0032_00.png', 'frame_0032_01.png', 'frame_0033_00.png', 'frame_0033_01.png', 'frame_0034_00.png', 'frame_0035_00.png', 'frame_0036_00.png', 'frame_0037_00.png', 'frame_0038_00.png', 'frame_0039_00.png', 'frame_0040_00.png', 'frame_0041_00.png', 'frame_0042_00.png', 'frame_0043_00.png', 'frame_0044_00.png', 'frame_0045_00.png', 'frame_0046_00.png', 'frame_0047_00.png', 'frame_0048_00.png', 'frame_0049_00.png', 'frame_0050_00.png', 'frame_0051_

## Execution of Person Classification

In [16]:
# Loading of interviewer and interviewee names
interviewee_name = ""

if dir_chosen in interviewee_data.keys():
    interviewee_name = interviewee_data[dir_chosen]

interviewee_name_lc = interviewee_name.lower()
interviewee_name_ns = interviewee_name_lc.replace(" ", "-")

if not os.path.exists(f'reference_imgs/interviewees/{interviewee_name_ns}.png'):
    reference_img_interviewee = f'reference_imgs/interviewees/{interviewee_name_ns}.jpg'
else:
    reference_img_interviewee = f'reference_imgs/interviewees/{interviewee_name_ns}.png'

if not os.path.exists(f'reference_imgs/interviewees/{interviewee_name_ns}-side.png'):
    reference_img_interviewee_side = f'reference_imgs/interviewees/{interviewee_name_ns}-side.jpg'
else:
    reference_img_interviewee_side = f'reference_imgs/interviewees/{interviewee_name_ns}-side.png'

if not os.path.exists(f'reference_imgs/interviewees/{interviewee_name_ns}-side-flipped.png'):
    reference_img_interviewee_side_flipped = f'reference_imgs/interviewees/{interviewee_name_ns}-side-flipped.jpg'
else:
    reference_img_interviewee_side_flipped = f'reference_imgs/interviewees/{interviewee_name_ns}-side-flipped.png'

print(reference_img_interviewee, "-", os.path.exists(reference_img_interviewee))
print(reference_img_interviewee_side, "-", os.path.exists(reference_img_interviewee_side))
print(reference_img_interviewee_side_flipped, "-", os.path.exists(reference_img_interviewee_side_flipped))

interviewer_name = ""

if dir_chosen in interviewer_data.keys():
    interviewer_name = interviewer_data[dir_chosen]

interviewer_name_lc = interviewer_name.lower()
interviewer_name_ns = interviewer_name_lc.replace(" ", "-")

reference_img_interviewer = ''
reference_img_interviewer_side = ""
reference_img_interviewer_side_flipped = ""

if not os.path.exists(f'reference_imgs/interviewer/{interviewer_name_ns}.png'):
    reference_img_interviewer = f'reference_imgs/interviewer/{interviewer_name_ns}.jpg'
else:
    reference_img_interviewer = f'reference_imgs/interviewer/{interviewer_name_ns}.png'

if not os.path.exists(f'reference_imgs/interviewer/{interviewer_name_ns}-side.png'):
    reference_img_interviewer_side = f'reference_imgs/interviewer/{interviewer_name_ns}-side.jpg'
else:
    reference_img_interviewer_side = f'reference_imgs/interviewer/{interviewer_name_ns}-side.png'

if not os.path.exists(f'reference_imgs/interviewer/{interviewer_name_ns}-side-flipped.png'):
    reference_img_interviewer_side_flipped = f'reference_imgs/interviewer/{interviewer_name_ns}-side-flipped.jpg'
else:
    reference_img_interviewer_side_flipped = f'reference_imgs/interviewer/{interviewer_name_ns}-side-flipped.png'

print(reference_img_interviewer, "-", os.path.exists(reference_img_interviewer))
print(reference_img_interviewer_side, "-", os.path.exists(reference_img_interviewer_side))
print(reference_img_interviewer_side_flipped, "-", os.path.exists(reference_img_interviewer_side_flipped))

reference_imgs/interviewees/raphael-pace.png - True
reference_imgs/interviewees/raphael-pace-side.png - True
reference_imgs/interviewees/raphael-pace-side-flipped.png - True
reference_imgs/interviewer/jon-mallia.jpg - True
reference_imgs/interviewer/jon-mallia-side.jpg - True
reference_imgs/interviewer/jon-mallia-side-flipped.jpg - True


In [73]:
if roi_list_exists_in_file:
    # Initialise person category dictionary if empty
    if not bool(video_person_categorized):
        for person_category in ["interviewer", "interviewee", "unknown"]:
            video_person_categorized[person_category] = []

    for snapshot_img in os.listdir(dir_chosen_full_path):
        #if os.path.isfile(snapshot_img):
        if snapshot_img.endswith('.jpg') or snapshot_img.endswith('.png'):
            base_img_name = os.path.splitext(snapshot_img)[0][:-3]
            snapshot_path = os.path.join(dir_chosen_full_path, snapshot_img)
            try:
                with suppress_stdout():
                    #snapshot_encodings = DeepFace.represent(snapshot_path, model_name='ArcFace', detector_backend='mtcnn')
                    #snapshot_encodings = DeepFace.represent(snapshot_path, model_name='VGG-Face', detector_backend='mtcnn')
                    snapshot_encodings = DeepFace.represent(snapshot_path, model_name='Facenet512', detector_backend='mtcnn')
            except:
                print(f'No face detected in {snapshot_img}!')
                video_person_categorized["unknown"].append(snapshot_img)
                write_person_detected(chosen_video_data_path, base_img_name, os.path.splitext(snapshot_img)[0], "unknown")
                continue
            
            try:
                category = "Unknown"
                #category, conf_score  = compare_person_new(snapshot_path, [reference_img_interviewer, reference_img_interviewer_side, reference_img_interviewer_side_flipped], [reference_img_interviewee, reference_img_interviewee_side, reference_img_interviewee_side_flipped], facial_rec_model='ArcFace', custom_threshold=1.2)
                #category, conf_score = compare_person_new(snapshot_path, [reference_img_interviewer, reference_img_interviewer_side, reference_img_interviewer_side_flipped], [reference_img_interviewee, reference_img_interviewee_side, reference_img_interviewee_side_flipped], facial_rec_model='VGG-Face', custom_threshold=1.2)
                category, conf_score  = compare_person_new(snapshot_path, [reference_img_interviewer, reference_img_interviewer_side, reference_img_interviewer_side_flipped], [reference_img_interviewee, reference_img_interviewee_side, reference_img_interviewee_side_flipped], facial_rec_model='Facenet512', custom_threshold=1.2)
                

                if category == "Interviewee":
                    if interviewee_name != "":
                        print(f"{interviewee_name} (Interviewee) is detected in {snapshot_img}")
                    else:
                        print(f"Interviewee is detected in {snapshot_img}")
                elif category == "Interviewer":
                    if interviewer_name != "":
                        print(f"{interviewer_name} (Interviewer) is detected in {snapshot_img}")
                    else:
                        print(f"Interviewer is detected in {snapshot_img}")
                else:
                    print(f"Face not recognized in {snapshot_img} — person not found in the database.")
                
                video_person_categorized[category.lower()].append(snapshot_img)
                write_person_detected(chosen_video_data_path, base_img_name, os.path.splitext(snapshot_img)[0], category.lower())
                if category != "Unknown":
                    write_person_score(chosen_video_data_path, base_img_name, os.path.splitext(snapshot_img)[0], conf_score)
                #print(f"{snapshot_img} belongs to {category}")
            except Exception as e:
                print(f"Error processing {snapshot_img}: {e}")
    print(f"Person Classification process for {dir_chosen} has finished!")
else:
    raise Exception(f'Please ensure that all facial ROIs from {dir_chosen} have been extracted before running this!')

Raphael Pace (Interviewee) is detected in frame_0009_00.png
Raphael Pace (Interviewee) is detected in frame_0010_00.png
Raphael Pace (Interviewee) is detected in frame_0011_00.png
Raphael Pace (Interviewee) is detected in frame_0012_00.png
Raphael Pace (Interviewee) is detected in frame_0013_00.png
Raphael Pace (Interviewee) is detected in frame_0014_00.png
Jon Mallia (Interviewer) is detected in frame_0015_00.png
Jon Mallia (Interviewer) is detected in frame_0016_00.png
Raphael Pace (Interviewee) is detected in frame_0016_01.png
Jon Mallia (Interviewer) is detected in frame_0017_00.png
Raphael Pace (Interviewee) is detected in frame_0018_00.png
Raphael Pace (Interviewee) is detected in frame_0019_00.png
Jon Mallia (Interviewer) is detected in frame_0020_00.png
Raphael Pace (Interviewee) is detected in frame_0020_01.png
Raphael Pace (Interviewee) is detected in frame_0021_00.png
Jon Mallia (Interviewer) is detected in frame_0021_01.png
Jon Mallia (Interviewer) is detected in frame_0022

### Loading results

In [17]:
video_person_categorized = load_person_categories(chosen_video_data_path)

if 'unknown' not in video_person_categorized.keys():
    video_person_categorized['unknown'] = []

for person_category, cat_list in video_person_categorized.items():
    print(person_category, '-', cat_list)

interviewee - ['frame_0009_00.png', 'frame_0010_00.png', 'frame_0011_00.png', 'frame_0012_00.png', 'frame_0013_00.png', 'frame_0014_00.png', 'frame_0016_01.png', 'frame_0018_00.png', 'frame_0019_00.png', 'frame_0020_01.png', 'frame_0021_00.png', 'frame_0023_00.png', 'frame_0027_00.png', 'frame_0028_00.png', 'frame_0029_00.png', 'frame_0030_00.png', 'frame_0032_01.png', 'frame_0033_01.png', 'frame_0037_00.png', 'frame_0038_00.png', 'frame_0039_00.png', 'frame_0040_00.png', 'frame_0042_00.png', 'frame_0043_00.png', 'frame_0045_00.png', 'frame_0047_00.png', 'frame_0048_00.png', 'frame_0049_00.png', 'frame_0050_00.png', 'frame_0051_00.png', 'frame_0052_00.png', 'frame_0053_00.png', 'frame_0055_00.png', 'frame_0056_00.png', 'frame_0057_00.png', 'frame_0058_00.png', 'frame_0060_00.png', 'frame_0061_00.png', 'frame_0062_00.png', 'frame_0063_00.png', 'frame_0064_00.png', 'frame_0067_00.png', 'frame_0068_00.png', 'frame_0069_00.png', 'frame_0070_00.png', 'frame_0071_00.png', 'frame_0072_00.png'

## Execution of Emotion Classification

In [20]:
if roi_list_exists_in_file:
    if not bool(video_person_categorized):
        video_person_categorized = load_person_categories(chosen_video_data_path)
        if 'unknown' not in video_person_categorized.keys():
            video_person_categorized['unknown'] = []
        
    for snapshot_img in os.listdir(dir_chosen_full_path)[0:30]:
        if snapshot_img.endswith('.jpg') or snapshot_img.endswith('.png'):
            if snapshot_img not in video_person_categorized['unknown']:
                base_img_name = os.path.splitext(snapshot_img)[0][:-3]
                snapshot_path = os.path.join(dir_chosen_full_path, snapshot_img)
                try:
                    with suppress_stdout():
                        snapshot_encodings = DeepFace.represent(snapshot_path, detector_backend='mtcnn')
                except:
                    print(f'No face detected in {snapshot_img}!')
                    continue
                
                result = ''
                if len(snapshot_encodings) == 1:
                    emotion_displayed, emotion_probs, entropy_value = predict_emotion(snapshot_path, snapshot_img, video_emotions_categorized, backend_detector='retinaface')
                else:
                    highest_detected_confidence = -0.01
                    best_encoding = None
                    for encoding_index, encoding in enumerate(snapshot_encodings):
                        if encoding['face_confidence'] > highest_detected_confidence:
                            highest_detected_confidence = encoding['face_confidence']
                            best_encoding = encoding
                    
                    snapshot_cv = cv2.imread(snapshot_path)

                    roi_x = best_encoding['facial_area']['x']
                    roi_y = best_encoding['facial_area']['y']
                    roi_w = best_encoding['facial_area']['w']
                    roi_h = best_encoding['facial_area']['h']
                    roi_left, roi_right = roi_x, roi_x + roi_w
                    roi_top, roi_bottom = roi_y, roi_y + roi_h

                    if roi_top < 0:
                        roi_top = 0
                    
                    if roi_left < 0:
                        roi_left = 0

                    snapshot_crop = snapshot_cv[roi_top:roi_bottom, roi_left:roi_right]
                    emotion_displayed, emotion_probs, entropy_value = predict_emotion(snapshot_crop, snapshot_img, video_emotions_categorized, backend_detector='retinaface')
                arousal_val, valence_val = calculate_av(emotion_probs)
                print(f'Emotion displayed in {snapshot_img}: {emotion_displayed}')
                #print(f"Emotion Probabilties: {emotion_probs}")
                #print(f"Entropy for {snapshot_img}: {entropy_value}")
                write_emotion_detected(chosen_video_data_path, base_img_name, os.path.splitext(snapshot_img)[0], emotion_displayed.lower())
                write_emotion_probs(chosen_video_data_path, base_img_name, os.path.splitext(snapshot_img)[0], emotion_probs)
                write_emotion_entropy(chosen_video_data_path, base_img_name, os.path.splitext(snapshot_img)[0], entropy_value)
                write_av_values(chosen_video_data_path, base_img_name, os.path.splitext(snapshot_img)[0], arousal_val, valence_val)
    print(f"Emotion Classification process for {dir_chosen} has finished!")
else:
    raise Exception(f'Please ensure that all facial ROIs from {dir_chosen} have been extracted before running this!')

Emotion displayed in frame_0009_00.png: neutral
Emotion displayed in frame_0010_00.png: happy
Emotion displayed in frame_0011_00.png: sad
Emotion displayed in frame_0012_00.png: angry
Emotion displayed in frame_0013_00.png: sad
Emotion displayed in frame_0014_00.png: neutral
Emotion displayed in frame_0015_00.png: sad
Emotion displayed in frame_0016_00.png: sad
Emotion displayed in frame_0016_01.png: angry
Emotion displayed in frame_0017_00.png: neutral
Emotion displayed in frame_0018_00.png: happy
Emotion displayed in frame_0019_00.png: neutral
Emotion displayed in frame_0020_00.png: angry
Emotion displayed in frame_0020_01.png: angry
Emotion displayed in frame_0021_00.png: sad
Emotion displayed in frame_0021_01.png: angry
Emotion displayed in frame_0022_00.png: neutral
Emotion displayed in frame_0023_00.png: sad
Emotion displayed in frame_0024_00.png: sad
Emotion displayed in frame_0025_00.png: neutral
Emotion displayed in frame_0026_00.png: neutral
Emotion displayed in frame_0027_00

In [21]:
video_emotion_categorized = load_emotion_categories(chosen_video_data_path)

for emotion_category, cat_list in video_emotion_categorized.items():
    print(emotion_category, '-', cat_list)

neutral - ['frame_0009_00.png', 'frame_0014_00.png', 'frame_0017_00.png', 'frame_0019_00.png', 'frame_0022_00.png', 'frame_0025_00.png', 'frame_0026_00.png', 'frame_0028_00.png', 'frame_0034_00.png', 'frame_0035_00.png', 'frame_0036_00.png', 'frame_0037_00.png', 'frame_0039_00.png', 'frame_0043_00.png', 'frame_0044_00.png', 'frame_0053_00.png', 'frame_0056_00.png', 'frame_0057_00.png', 'frame_0058_01.png', 'frame_0062_00.png', 'frame_0066_00.png', 'frame_0068_00.png', 'frame_0073_00.png', 'frame_0075_00.png', 'frame_0078_00.png', 'frame_0080_00.png', 'frame_0086_00.png', 'frame_0089_00.png', 'frame_0091_00.png', 'frame_0092_00.png', 'frame_0093_00.png', 'frame_0098_00.png', 'frame_0102_00.png', 'frame_0105_00.png', 'frame_0106_00.png', 'frame_0109_00.png', 'frame_0110_00.png', 'frame_0112_00.png', 'frame_0113_00.png', 'frame_0117_00.png', 'frame_0120_00.png', 'frame_0124_00.png', 'frame_0125_00.png', 'frame_0127_00.png', 'frame_0129_00.png', 'frame_0130_00.png', 'frame_0131_00.png', 'f